![header](../figures/logos_partenaires._cmems_se.jpg)

# Gulf Stream: Benchmark DUACS plot maps
 

***
**Authors:**  CLS & Datlas <br>
**Copyright:** 2023 CLS & Datlas <br>
**License:** MIT

<div class="alert alert-block alert-success">
<h1><center>Gulf Stream: Benchmark of DUACS plot maps</center></h1>
<h5> The notebook aims to plot maps produced by the DUACS system. </h5>
    <h5> These maps are equivalent to the SEALEVEL_GLO_PHY_L4_MY_008_047 product distributed by the Copernicus Marine Service, except that a nadir altimeter (SARAL/Altika, SEALEVEL_GLO_PHY_L3_MY_008_062 product) has been excluded from the mapping. </h5>
        <h5> We provide below a demonstration of the validation of these maps against the independent SSH data from the Saral/AltiKa altimeter distributed by CMEMS </h5>
</div>

***
**General Note 1**: Execute each cell through the <button class="btn btn-default btn-xs"><i class="icon-play fa fa-play"></i></button> button from the top MENU (or keyboard shortcut `Shift` + `Enter`).<br>
<br>
**General Note 2**: If, for any reason, the kernel is not working anymore, in the top MENU, click on the <button class="btn btn-default btn-xs"><i class="fa fa-repeat icon-repeat"></i></button> button. Then, in the top MENU, click on "Cell" and select "Run All Above Selected Cell".<br>
***


<div class="alert alert-danger" role="alert">

<h3>Learning outcomes</h3>

At the end of this notebook you will know:
<ul>
  <li>How you can evaluated sea surface height maps with independent alongtrack data: statistical and spectral analysis</li>
</ul>
    
</div>

In [1]:
from glob import glob
import numpy as np
import os

In [2]:
import sys
sys.path.append('..')
from src.mod_plot import *
from src.mod_stat import *
from src.mod_spectral import *
from src.mod_interp import *
from src.mod_switchvar import *

In [3]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

<div class="alert alert-info" role="alert">

<h2>0. Parameters</h2>

</div>

In [51]:
time_min = '2019-01-01'                                        # time min for analysis
time_max = '2019-02-01'                                        # time max for analysis
output_dir = '../results'                                      # output directory path
os.system(f'mkdir -p {output_dir}')

# Region details
region = 'GS'
lon_min = -65                                          # domain min longitude
lon_max = -55                                          # domain max longitude
lat_min = 33.                                          # domain min latitude
lat_max = 43.                                          # domain max latitude 
box_lonlat = {'lon_min':lon_min,'lon_max':lon_max,'lat_min':lat_min,'lat_max':lat_max}

method_name = 'BFNQG'



<div class="alert alert-info" role="alert">

<h2>1. Input files</h2>

</div>

## Sea Surface Height maps to evaluate

In [52]:
%%time
list_of_maps = sorted(glob('../data/maps/DC_MapGlob_GSZoom_BFNQG/DC*.nc'))
ds_maps = xr.open_mfdataset(list_of_maps, combine='nested', concat_dim='time')
ds_maps = ds_maps.sel(time=slice(time_min, time_max))
ds_maps

CPU times: user 323 ms, sys: 81.2 ms, total: 404 ms
Wall time: 475 ms


<xarray.Dataset>
Dimensions:  (time: 32, lat: 102, lon: 102)
Coordinates:
  * time     (time) datetime64[ns] 2019-01-01 2019-01-02 ... 2019-02-01
  * lon      (lon) float64 295.0 295.1 295.2 295.3 ... 304.8 304.9 305.0 305.1
  * lat      (lat) float64 33.0 33.1 33.2 33.3 33.4 ... 42.7 42.8 42.9 43.0 43.1
Data variables:
    sla      (time, lat, lon) float64 dask.array<chunksize=(1, 102, 102), meta=np.ndarray>
    pv       (time, lat, lon) float64 dask.array<chunksize=(1, 102, 102), meta=np.ndarray>

## Selecting only Gulf Stream

In [53]:

from src.mod_compare import regional_zoom

ds_maps = ds_maps.rename({'lon':'longitude','lat':'latitude'})
ds_maps = regional_zoom(ds_maps, [box_lonlat['lon_min'],box_lonlat['lon_max']], [box_lonlat['lat_min'],box_lonlat['lat_max']], namelon='longitude', namelat='latitude', change_lon=True)

ds_maps.sla.attrs = {'grid_mapping': 'crs',
                     'long_name': 'Sea level anomaly',
                     'standard_name': 'sea_surface_height_above_sea_level',
                     'units': 'm'}

In [54]:
ds_maps

<xarray.Dataset>
Dimensions:    (time: 32, latitude: 99, longitude: 99)
Coordinates:
  * time       (time) datetime64[ns] 2019-01-01 2019-01-02 ... 2019-02-01
  * longitude  (longitude) float64 -64.9 -64.8 -64.7 -64.6 ... -55.3 -55.2 -55.1
  * latitude   (latitude) float64 33.1 33.2 33.3 33.4 ... 42.6 42.7 42.8 42.9
Data variables:
    sla        (time, latitude, longitude) float64 dask.array<chunksize=(1, 99, 99), meta=np.ndarray>
    pv         (time, latitude, longitude) float64 dask.array<chunksize=(1, 99, 99), meta=np.ndarray>

## Plot BFN-QG SSH movie

In [19]:
 
movie(ds_maps, 'sla', method=method_name, region=region, dir_output='../results/',dim_name=['time','latitude','longitude'],
          framerate=24,Display=True,clim=None,cmap='Spectral', newmovie=True)


## Plot BFN-QG geostrophic currents movie

In [60]:
ds_maps

<xarray.Dataset>
Dimensions:    (time: 32, latitude: 99, longitude: 99)
Coordinates:
  * time       (time) datetime64[ns] 2019-01-01 2019-01-02 ... 2019-02-01
  * longitude  (longitude) float64 -64.9 -64.8 -64.7 -64.6 ... -55.3 -55.2 -55.1
  * latitude   (latitude) float64 33.1 33.2 33.3 33.4 ... 42.6 42.7 42.8 42.9
Data variables:
    sla        (time, latitude, longitude) float64 dask.array<chunksize=(1, 99, 99), meta=np.ndarray>
    pv         (time, latitude, longitude) float64 dask.array<chunksize=(1, 99, 99), meta=np.ndarray>

In [66]:
ds_maps = sla_to_ssh(ds_maps,'/Users/sammymetref/Documents/DATLAS/Data/cnes_obs-sl_glo_phy-mdt_my_0.125deg_P20Y_1695393893725_1mdt.nc')

u,v = ssh_to_currents(ds_maps.ssh, ds_maps.longitude, ds_maps.latitude)

ds_maps = ds_maps.assign(ugos=lambda ds_maps: ds_maps.sla*0  + u)
ds_maps.ugos.attrs = {'grid_mapping': 'crs',
                    'long_name': 'Absolute geostrophic velocity: zonal component',
                    'standard_name': 'surface_geostrophic_eastward_sea_water_velocity',
                    'units': 'm/s'}

ds_maps = ds_maps.assign(vgos=lambda ds_maps: ds_maps.sla*0  + v)
ds_maps.vgos.attrs = {'grid_mapping': 'crs',
                    'long_name': 'Absolute geostrophic velocity: meridian component',
                    'standard_name': 'surface_geostrophic_northward_sea_water_velocity',
                    'units': 'm/s'}

<xarray.DataArray 'longitude' ()>
array(-64.9)
<xarray.DataArray 'longitude' ()>
array(-55.1)
<xarray.DataArray 'latitude' ()>
array(33.1)
<xarray.DataArray 'latitude' ()>
array(42.9)
<xarray.Dataset>
Dimensions:    (longitude: 80, latitude: 80)
Coordinates:
  * latitude   (latitude) float32 33.06 33.19 33.31 33.44 ... 42.69 42.81 42.94
    time       datetime64[ns] 2003-01-01
  * longitude  (longitude) float32 -64.94 -64.81 -64.69 ... -55.31 -55.19 -55.06
Data variables:
    crs        (longitude, latitude) float64 -2.147e+09 ... -2.147e+09
    mdt        (latitude, longitude) float64 0.6111 0.6085 ... -0.175 -0.1726
Attributes: (12/40)
    cdm_data_type:                                  Grid
    history:                                        2020-07-29T10:52:01Z : Cr...
    Conventions:                                    CF-1.6
    FROM_ORIGINAL_FILE__Metadata_Conventions:       Unidata Dataset Discovery...
    contact:                                        servicedesk.cmems@mercat

In [73]:
movie(ds_maps,name_var='uv', method=method_name, region=region, dir_output='../results/',dim_name=['time','latitude','longitude'],
          framerate=24,Display=True,clim=[0,1.2],cmap='YlGnBu_r', newmovie=True)


## Compute and plot DUACS relative vorticity movie

In [69]:
rv = currents_to_relative_vorticity(np.array(ds_maps.ugos),np.array(ds_maps.vgos), ds_maps.longitude,ds_maps.latitude)

ds_maps = ds_maps.assign(rv=lambda ds_maps: ds_maps.sla * 0 + rv)

In [75]:
movie(ds_maps,name_var='rv', method=method_name, region=region, dir_output='../results/',dim_name=['time','latitude','longitude'],
          framerate=24,Display=True,clim=[-0.8,0.8],cmap='RdBu_r', newmovie=True)
